In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

/home/hydra/Desktop/genAi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.0-flash",
    temperature=1.0,
    max_retries=2,
    google_api_key=google_api_key,
)

In [31]:
# @tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=07b88a4b41c119c16b6775f49dbd6e52={city}'

  response = requests.get(url)

  return response.json()

In [32]:
get_weather_data('attock')

{'success': False,
 'error': {'code': 101,
  'type': 'invalid_access_key',
  'info': 'You have not supplied a valid API Access Key. [Technical Support: support@apilayer.com]'}}

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Set up DuckDuckGo search for news
search_tool = DuckDuckGoSearchRun(
    api_wrapper=DuckDuckGoSearchAPIWrapper(source="news")
)


In [5]:
search_tool.invoke("uscis case status")  # Example usage of the search tool

'The H-1B visa applicants can now very easily check their case status online. This can be done through by visiting the official website of United States Citizenship and Immigration Services (USCIS). "Every immigration case entrusted to us represents an individual ... including the applications for EADs (Form I-765), adjustment of status (Form I-485), and naturalization (Form N-400)." USCIS also has vowed to improve asylum filing and adjudication ... Several EB-1A applicants allegedly manipulated the system by publishing in low-quality journals, generating fake citations through coordinated co-authorship, and promoting dubious awards such as the Stevie and Globee to bolster their profiles. An immigrant from the country of Georgia who has SIJ status and has been waiting ... the average time USCIS has taken to approve a case is 263 days, which, although an improvement, is still ... The majority were between the ages of 19 and 22 when they were arrested, according to the latest available d

In [6]:
llm.invoke('hi').content

'Hi there! How can I help you today?'

In [7]:
# for agent we need create agent and agent executor from langchain and for prompt we need ReAct prompt from langchain.hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [8]:
# stp2 pull the ReAct prompt from langchain hub
react_prompt = hub.pull("hwchase17/react")

/home/hydra/Desktop/genAi/lib/python3.12/site-packages/langsmith/client.py:271: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
react_prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [10]:
#  stp 3 create the agent
agent = create_react_agent(
    llm=llm,
    prompt=react_prompt,
    tools=[search_tool],
)


In [19]:
# stp 4 create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
)

In [30]:
report = agent_executor.invoke({"input": HumanMessage("what is the latest news about iran and israel")})




> Entering new AgentExecutor chain...
I need to find the latest news about Iran and Israel. I will use duckduckgo_search to find recent articles.
Action: duckduckgo_search
Action Input: "iran israel news"WASHINGTON − President Donald Trump denied reports he had reached out to Tehran seeking an end to the five-day Iran-Israel air war, after earlier suggesting he was working toward a broad Iran nuclear deal. Israel stunned Iran last week with an intelligence and military operation years in the making that struck high-level targets with precision. President Donald Trump said overnight Tuesday he's seeking "a real end" to the conflict between Israel and Iran, arguing that outcome would be preferable to a ceasefire while also suggesting his patience for negotiations was wearing thin. Israel said Tuesday that they killed Iran's wartime chief of staff Maj. Gen. Ali Shamdani, a close ally of Iranian Supreme Leader Ayatollah Ali Khamenei. Aerial attacks between Israel and Iran continued overn

In [31]:
report

{'input': HumanMessage(content='what is the latest news about iran and israel', additional_kwargs={}, response_metadata={}),
 'output': 'Recent news indicates an escalation of conflict between Iran and Israel, involving aerial attacks, intelligence operations, and reports of strikes on key facilities and the killing of Iranian military leaders. Discussions about negotiations and a potential nuclear deal are also ongoing.'}